# Social network 3: Complex and dynamical networks using Python and networkx

In this workshop, we will focus on generating networks through code, and seeing some of the interesting and at times unexpected dynamics of networks.

We will learn how to generate network using algorithms, and how the algorithms shape the properties of these networks


# 1. Small world simulation

In this exercise, we will start with a geographical network, in which each node is connected to their four closest neighbhors, in a big circle.

We will then see how the average shortest path depends on the number of edges we randomize in this network.

The aim of this exercise is to see how nearly universal the small-world phenomenon is. 

In [98]:
# Circle network
import networkx as nx
from matplotlib import pyplot as plt

def create_circle_network(num_nodes=1000):
    # This is code creates a simple circular network, were each node is connected to their neighbors 
    # Create an empty graph
    G = nx.Graph()
    
    # Add nodes and edges to create a circular network
    for i in range(num_nodes):
        G.add_node(i)
        G.add_edge(i, (i + 1) % num_nodes)  # Connect each node to its neighbor, using modulo for circular connection
        G.add_edge(i, (i + 2) % num_nodes)  # Connect each node to its second neighbor, using modulo for circular connection
    
    # Draw the network
    nx.draw_circular(G, with_labels=True, node_color='lightblue', node_size=20, font_size=5)
    plt.title(f"Random Network with {num_nodes} Nodes")
    plt.show()
    return G

In [332]:
G = create_circle_network()
print(f'The average shortest path between two nodes in this network is {nx.average_shortest_path_length(G)}')

### Question:
Your task is to answer: *What fraction of edges would you need to replace with random edges for the average shortest path to drop to 1/10 of this value* (i.e. 12.5)? Write code to replace the edges one by one by a random edge. For each edge removed, measure the new average shortest path. Plot the average shortest path as a function of the fraction of edges that are randomized. How do you interpret your finding? 


# 2. Modeling how we make friends

Let's think about different models for how we make friends, and examine the type of social network that this results in.

### Model 1: We meet random people
One is just by meeting people by random. This would suggest just adding edges between two randomly selected nodes until we have as many edges that we want.

This is called an Erdos-Renyi network.

#### Your task: 
1. Write code for a network in which the nodes are connected to other nodes by random.
2. Plot the degree distribution. You have come across this distribution before. What type of distribution is it? 

In [327]:
# Random network
import networkx as nx
from matplotlib import pyplot as plt
import itertools
import numpy as np

# This code creates a network where nodes are 
def create_random_network(num_nodes,num_edges_per_node):
    G = nx.Graph()
    
    #Add the nodes
    for i in range(num_nodes):
        G.add_node(i)
    
    for i in range(num_edges_per_node*num_nodes):
        ### [YOUR CODE HERE] ###
    
    # Draw the network
    nx.draw(G, with_labels=True, node_color='lightblue', node_size=20, font_size=5)
    plt.title(f"Random Network with {num_nodes} Nodes")
    plt.show()
    return G

In [331]:
# Feel free to vary the parameters
G = create_random_network(1000,50)

Let's plot the degree distribution of this network. To do so, we create a histogram of the frequency of each number of degrees. 

The degree distribution is a highly important measure, as it shows how equal the network is, and how dominated it is by a few nodes.



In [330]:
# Plot the degree distribution 

import networkx as nx
import matplotlib.pyplot as plt

def plot_degree_distribution(G):

    # Calculate the degrees of each node    
    # and create a histogram showing the distribution of degrees
    # [YOUR CODE HERE]

    # Set labels and title
    plt.xlabel('Degree')
    plt.ylabel('Frequency')
    plt.title('Degree Distribution of the Network')
    
    # Show the plot
    plt.show()
    return degrees
    
degrees = plot_degree_distribution(G)

### Model 2: Friends make friends through friends

The network we created, the nodes are connected randomly based on the nodes. But in most real-world networks, you make connections through your existing connections. For instance, the probability that you follow someone on Twitter depends on how many followers the person already has, since you are likely to encounter them through their followers.

These degree distributions are ubiquitous, and characterize nearly all networks around us, including link between websites, flights connecting airports, and so on.

In most social networks, nodes build connections based on their existing connections. This is called "preferential attachment".

#### Your task:
1. You will simulate the growth of a social media network. The network will have N nodes. These join one at the time. At each step, add one new node, then sample M existing nodes based on their edges, and add edges to these. Add 10 edges per node.
2. Plot the resulting degree distribution. What type of distribution is this? What does it mean that networks have this type of structure?


In [166]:
def create_random_network_by_edges(num_nodes,num_edges_per_node):
    #We start with a seed network
    G = nx.Graph()
    G.add_node(0)
    G.add_node(1)
    G.add_edge(0,1)
    
    #[YOUR CODE HERE]
    #Add edges based on existing nodes
    
    # Draw the network
    nx.draw(G, with_labels=True, node_color='lightblue', node_size=20, font_size=5)
    plt.show()
    return G

G = create_random_network_by_edges(1000,10)
degrees = plot_degree_distribution(G)


# 3. Epidemics in networks 

Networks also allow us to model dynamic phenomena, such as the spread of ideas, habits, or disease through networks. 

The structure of the network is central to how these spread. For instance, if Twitter changes how followingship is made, this will affect the dynamics of virality on Twitter.

The same accounts to disease spread. The SIR model is a classic framework used in epidemiology to understand how diseases spread through populations. SIR stands for Susceptible, Infected, and Recovered, which are the three possible states of individuals in this model:

- Susceptible (S): Individuals who have not yet contracted the disease and are vulnerable to infection.
- Infected (I): Individuals who have contracted the disease and are capable of spreading it to susceptible individuals.
- Recovered (R): Individuals who have recovered from the disease and are no longer susceptible to it. In some models, 'Recovered' can also include individuals who have died from the disease, as they are no longer part of the infection cycle.

We are now going to look at how the spread of the disease depends on the network structure. 

In [303]:
import networkx as nx
import matplotlib.pyplot as plt
import random

def SIR_simulation(G, initial_infected, infection_prob, recovery_prob):
    # Initialize all nodes as susceptible
    nx.set_node_attributes(G, 'S', 'state')

    # Infect initial nodes
    for node in initial_infected:
        G.nodes[node]['state'] = 'I'

    S, I, R = [], [], []
    # for _ in range(steps):
    while(True):
        new_infected = []
        new_recovered = []

        # Spread the infection
        for node in G:
            if G.nodes[node]['state'] == 'I':
                neighbors = list(G.neighbors(node))
                for neighbor in neighbors:
                    if G.nodes[neighbor]['state'] == 'S' and random.random() < infection_prob:
                        new_infected.append(neighbor)

                # Recover process
                if random.random() < recovery_prob:
                    new_recovered.append(node)

        # Update the states
        for node in new_infected:
            G.nodes[node]['state'] = 'I'
        for node in new_recovered:
            G.nodes[node]['state'] = 'R'

        S.append(sum(1 for n in G if G.nodes[n]['state'] == 'S'))
        nr_infected = sum(1 for n in G if G.nodes[n]['state'] == 'I')
        I.append(nr_infected)
        R.append(sum(1 for n in G if G.nodes[n]['state'] == 'R'))
        if nr_infected == 0:
            break

    return S, I, R

def plot_network(G):
    # Colors for nodes
    colors = ['blue' if G.nodes[node]['state'] == 'S' else ('red' if G.nodes[node]['state'] == 'I' else 'green') for node in G]

    plt.figure(figsize=(8, 6))
    nx.draw(G, node_color=colors, node_size=50, with_labels=False)
    plt.show()

# Parameters for the SIR simulation
initial_infected_count = 5
infection_prob = 0.02
recovery_prob = 0.1

# Create different network types
G = nx.watts_strogatz_graph(1000, 10, 0.1)    

# Example run!
initial_infected = random.sample(list(G.nodes()), initial_infected_count)
S, I, R = SIR_simulation(G, initial_infected, infection_prob, recovery_prob)
plot_network(G)


## Your task

Read the code above and make sure you can explain it. 

Your task is to compare how well a disease spreads in different network structures. In which network does the disease spread fastest? In which networks does the disease spread to the most nodes?

In [ ]:
# Networks to compare

"Scale-Free Network": nx.barabasi_albert_graph(1000, 2),
"Small-World Network": nx.watts_strogatz_graph(1000, 4, 0.1),
"Random Network": nx.erdos_renyi_graph(1000, 0.05),
"Network with Communities": nx.connected_caveman_graph(10, 100)


# 4. Cascading failures in electricity grids

Cascading failures in electricity grids refer to a process where a failure in one part of the grid triggers a chain of failures throughout the system, leading to a large-scale power outage or blackout. The cascade often begins with a single failure or fault in one component of the grid, such as a transmission line, transformer, or generator. When a component fails, the electrical load it was carrying is redistributed to other parts of the grid. If this redistribution results in an excessive load on these components, they can also fail. Each subsequent failure puts additional strain on the system, causing more components to fail. This can create a domino effect, leading to widespread disconnections and outages.

You are working for the US government, and you are worried about possible attacks against the electricity grid. You want to find out how vulnerable the network is against attacks, so that you can address the vulnerability. To do so, you've build a simulation.


In [281]:
# Load network
def initialize_grid():
    G = nx.read_gexf('us_powergrid.gexf')
    return G

# Reset the grid to unfailed status
def reset_grid(G):
    for node in G.nodes:
        G.nodes[node]['current_load'] = G.nodes[node]['load']
        G.nodes[node]['current_capacity'] = G.nodes[node]['capacity']
        G.nodes[node]['failed'] = False

# Fail a particular node
def fail_node(G, neighbors_dict, node):
    # Fail a node and redistribute its load
    load_to_redistribute = G.nodes[node]['current_load']
    G.nodes[node]['current_load'] = 0
    G.nodes[node]['failed'] = True
    neighbors = neighbors_dict[node]
    load_distribution = load_to_redistribute / len(neighbors)
    for neighbor in neighbors:
        G.nodes[neighbor]['current_load'] += load_distribution

# This code cascades the failure througout the network
def cascade_failure(G, neighbors_dict):
    failed = 0
    overloaded = True
    while overloaded:
        overloaded = False
        for node in list(G.nodes):
            if not G.nodes[node]['failed'] and G.nodes[node]['current_load'] > G.nodes[node]['current_capacity']:
                failed += 1
                fail_node(G, neighbors_dict, node)
                overloaded = True
    return failed

#Example run

#Load graph 
G = initialize_grid()

# Precompute neighbor lists for speed
neighbors_dict = {node: list(G.neighbors(node)) for node in G.nodes}

reset_grid(G)

# Simulate initial failure
initial_failure = random.choice(list(G.nodes))
fail_node(G, neighbors_dict, initial_failure)

# Simulate cascading failure
failed_nodes = cascade_failure(G,neighbors_dict)
print(f"The cascade brought down {failed_nodes} nodes.")


The cascade brought down 2 nodes.


### Your task
Read the code above and make sure you understand it and can explain it. 

Your task is build on this code to examine: 

1. What is the distribution of collapses when the node is selected at random?

2. Which node results in the largest cascade? How many nodes are brought down when this is attacked?